In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import sys
oldsysstdout = sys.stdout
class flushfile():
    def __init__(self, f):
        self.f = f
    def __getattr__(self,name): 
        return object.__getattribute__(self.f, name)
    def write(self, x):
        self.f.write(x)
        self.f.flush()
    def flush(self):
        self.f.flush()
sys.stdout = flushfile(sys.stdout)

In [3]:
import json

In [4]:
with open('../train.json', 'r') as f:
    train = json.load(f)
with open('../test.json', 'r') as f:
    test = json.load(f)

In [5]:
vec = []
for i in train:
    [vec.append(j) for j in i['ingredients']]

In [6]:
cats = np.unique((np.array(vec)))

In [7]:
from scipy.sparse import csr_matrix

In [8]:
X = csr_matrix(map(lambda y: np.array(map(lambda x: 1 if x in y['ingredients'] else 0, cats), dtype='int8'), train))

In [9]:
X.shape

(39774, 6714)

In [10]:
target = map(lambda x: x['cuisine'], train)

In [11]:
b, y = np.unique(target, return_inverse=True)

In [12]:
test_sp = csr_matrix(map(lambda y: np.array(map(lambda x: 1 if x in y['ingredients'] else 0, cats), dtype='int8'), test))

In [13]:
from sklearn.cross_validation import train_test_split, StratifiedKFold

In [49]:
cv = StratifiedKFold(y, n_folds = 5, random_state = 0)

In [15]:
import xgboost as xgb

In [56]:
params = {"objective": ["multi:softmax"], 
          "booster": ["gbtree"],
          "nthread": [3],
          "eta": [0.5], 
          "max_depth": [3, 5, 10, 20], 
          "subsample": [0.5, 0.7, 0.9], 
          "colsample_bytree": [0.5, 0.7, 0.9],
          "num_class": [20],
          "lambda": [1., 10., 100.], 
          "alpha": [0., .1, 1.],
          "silent": [1]
         } 
num_trees = 5000

In [57]:
from itertools import product, izip, permutations
grid = [dict(izip(params, x)) for x in product(*params.itervalues())]

In [58]:
len(grid)

324

In [59]:
from sklearn.metrics import accuracy_score

In [ ]:
serch = []
ite = 1
for params in grid:
    print("Starting training {} params of {}".format(ite, 324))
    ite += 1
    accuracies = []
    for train_index, test_index in cv:
        X_train = X[train_index]
        X_test = X[test_index]
        y_train = y[train_index]
        y_test = y[test_index]
        
        X_t, X_v, y_t, y_v = train_test_split(X_train, y_train, test_size=0.1, random_state=0)
        
        dtrain = xgb.DMatrix(X_t, y_t)
        dvalid = xgb.DMatrix(X_v, y_v)
        dtest = xgb.DMatrix(X_test)
        watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
        gbm = xgb.train(params, dtrain, num_trees, evals=watchlist, early_stopping_rounds=50, verbose_eval=False)
        
        if hasattr(gbm, 'best_iteration'):
            preds = gbm.predict(dtest, ntree_limit = gbm.best_iteration).astype('int64')
        else:
            preds = gbm.predict(dtest).astype('int64')
        accuracies.append(accuracy_score(y_test, preds))
    print("Accuracy: {:.4f} +/- {:.4f}".format(np.mean(accuracies), np.std(accuracies)))
    serch.append((np.mean(accuracies), np.std(accuracies), params))

Starting training 1 params of 324


Will train until valid error hasn't decreased in 50 rounds.
Stopping. Best iteration:
[154]	train-merror:0.162941	valid-merror:0.234444

Will train until valid error hasn't decreased in 50 rounds.
Stopping. Best iteration:
[261]	train-merror:0.132954	valid-merror:0.231301

Will train until valid error hasn't decreased in 50 rounds.
Stopping. Best iteration:
[223]	train-merror:0.142303	valid-merror:0.230358

Will train until valid error hasn't decreased in 50 rounds.
Stopping. Best iteration:
[249]	train-merror:0.132542	valid-merror:0.236569

Will train until valid error hasn't decreased in 50 rounds.
Stopping. Best iteration:
[198]	train-merror:0.149874	valid-merror:0.234998



Accuracy: 0.7626 +/- 0.0040
Starting training 2 params of 324


Will train until valid error hasn't decreased in 50 rounds.
Stopping. Best iteration:
[260]	train-merror:0.182047	valid-merror:0.236644

Will train until valid error hasn't decreased in 50 rounds.
Stopping. Best iteration:
[318]	train-merror:0.170881	valid-merror:0.233815

Will train until valid error hasn't decreased in 50 rounds.
Stopping. Best iteration:
[261]	train-merror:0.181625	valid-merror:0.235387

Will train until valid error hasn't decreased in 50 rounds.
Stopping. Best iteration:
[282]	train-merror:0.175978	valid-merror:0.247251

Will train until valid error hasn't decreased in 50 rounds.
Stopping. Best iteration:
[487]	train-merror:0.143381	valid-merror:0.232171



Accuracy: 0.7562 +/- 0.0045
Starting training 3 params of 324


...

Will train until valid error hasn't decreased in 50 rounds.
Stopping. Best iteration:
[236]	train-merror:0.079078	valid-merror:0.224073

Will train until valid error hasn't decreased in 50 rounds.
Stopping. Best iteration:
[309]	train-merror:0.062723	valid-merror:0.226901

Will train until valid error hasn't decreased in 50 rounds.
Stopping. Best iteration:
[193]	train-merror:0.092646	valid-merror:0.223759

Will train until valid error hasn't decreased in 50 rounds.
Stopping. Best iteration:
[159]	train-merror:0.106948	valid-merror:0.233428

Will train until valid error hasn't decreased in 50 rounds.
Stopping. Best iteration:
[283]	train-merror:0.066576	valid-merror:0.227144



Accuracy: 0.7687 +/- 0.0049
Starting training 253 params of 324


Will train until valid error hasn't decreased in 50 rounds.
Stopping. Best iteration:
[173]	train-merror:0.170520	valid-merror:0.237272

Will train until valid error hasn't decreased in 50 rounds.
Stopping. Best iteration:
[257]	train-merror:0.148355	valid-merror:0.231930

Will train until valid error hasn't decreased in 50 rounds.
Stopping. Best iteration:
[255]	train-merror:0.149532	valid-merror:0.230673

Will train until valid error hasn't decreased in 50 rounds.
Stopping. Best iteration:
[303]	train-merror:0.136767	valid-merror:0.238454

Will train until valid error hasn't decreased in 50 rounds.
Stopping. Best iteration:
[205]	train-merror:0.158532	valid-merror:0.238454



Accuracy: 0.7577 +/- 0.0024
Starting training 254 params of 324


Will train until valid error hasn't decreased in 50 rounds.
Stopping. Best iteration:
[360]	train-merror:0.171498	valid-merror:0.234444

Will train until valid error hasn't decreased in 50 rounds.
Stopping. Best iteration:
[415]	train-merror:0.161766	valid-merror:0.238215

Will train until valid error hasn't decreased in 50 rounds.
Stopping. Best iteration:
[261]	train-merror:0.187596	valid-merror:0.239786

Will train until valid error hasn't decreased in 50 rounds.
Stopping. Best iteration:
[341]	train-merror:0.172346	valid-merror:0.249450

